## Linear regression &  Random Forest

This notebook is expansion of Bhanu.Vijayaraj's analysis notebook.
To do regression analysis it was added:
1. data transformation (data types, one-hote features encode, datasets merge, filterin data)
2. model training

In [1]:
import os
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import eli5 
from sklearn.metrics import r2_score, mean_absolute_error,mean_squared_error

In [4]:
add_data = pd.read_csv(r'games_list_add_data.csv', sep = '\t', encoding='utf-8',nrows=1000)

In [5]:
add_data.drop(['url','full_url','overview_url'],axis=1,inplace=True)

In [6]:
add_data

,name,id,play_time,player_age,user_players
0,Gloomhaven,174430,60–120,14+,3.0
1,Pandemic Legacy: Season 1,161936,60,12+,4.0
2,Brass: Birmingham,224517,60–120,14+,3–4
3,Terraforming Mars,167791,120,12+,3
4,Twilight Imperium: Fourth Edition,233078,240–480,14+,6
...,...,...,...,...,...
995,Reavers of Midgard,242684,60–120,12+,4
996,GIPF,527,30–60,8+,2
997,Agra,230085,90–120,14+,4
998,Mission: Red Planet,18258,60,10+,4–5


In [7]:
play_time = add_data['play_time'].value_counts()

In [8]:
original_play_time = add_data['play_time']
avg_play_time = []
for index, value in enumerate(original_play_time):
    temp_split_arr = value.split("–")
    r = []
    for index, value in enumerate(temp_split_arr):
        r.append(int(value))
    temp_avg = np.mean(r)
    avg_play_time.append(temp_avg)
avg_play_time
len(avg_play_time)

1000

In [9]:
player_age = add_data['player_age'].value_counts()

In [10]:
original_player_age = add_data['player_age']
min_play_age = []
for index, value in enumerate(original_player_age):
    temp_split_arr = value.split("+")
    for index, value in enumerate(temp_split_arr):
        if index == 0:
          temp_value = int(value)
          min_play_age.append(temp_value)
len(min_play_age)

1000

In [11]:
original_num_player = add_data['user_players']
min_num_player = []
max_num_player = []
temp_plus = []
for index, value in enumerate(original_num_player):
    temp_split_arr = []
    if "none" in value:
        min_num_player.append(2)
        max_num_player.append(2)
    elif "–" in value:
        temp_split_arr = value.split("–")
        min_num_player.append(int(temp_split_arr[0]))
        if "+" in temp_split_arr[1]:
            temp_plus = temp_split_arr[1].split("+")
            max_num_player.append(int(temp_plus[0]))
        else:
            max_num_player.append(int(temp_split_arr[1]))
    elif "," in value:
        temp_split_arr = value.split(",")
        l = len(temp_split_arr)
        min_num_player.append(int(temp_split_arr[0]))
        max_num_player.append(int(temp_split_arr[l-1]))
    elif "+" in value:
        temp_split_arr = value.split("+")
        min_num_player.append(int(temp_split_arr[0]))
        max_num_player.append(int(temp_split_arr[0]))
    elif "." in value:
        temp_split_arr = value.split(".")
        min_num_player.append(int(temp_split_arr[0]))
        max_num_player.append(int(temp_split_arr[0]))
    else:
        min_num_player.append(value)
        max_num_player.append(value)

In [12]:
games_df = pd.read_csv(r'games_list_add_data.csv', sep = '\t', encoding='utf-8',nrows=1000)
games_df.drop(['url','full_url','overview_url'],axis=1,inplace=True)
games_df['avg_play_time'] = avg_play_time
games_df['min_player_age'] = min_play_age
games_df['min_num_player'] = min_num_player
games_df['max_num_player'] = max_num_player
games_df

,name,id,play_time,player_age,user_players,avg_play_time,min_player_age,min_num_player,max_num_player
0,Gloomhaven,174430,60–120,14+,3.0,90.0,14,3,3
1,Pandemic Legacy: Season 1,161936,60,12+,4.0,60.0,12,4,4
2,Brass: Birmingham,224517,60–120,14+,3–4,90.0,14,3,4
3,Terraforming Mars,167791,120,12+,3,120.0,12,3,3
4,Twilight Imperium: Fourth Edition,233078,240–480,14+,6,360.0,14,6,6
...,...,...,...,...,...,...,...,...,...
995,Reavers of Midgard,242684,60–120,12+,4,90.0,12,4,4
996,GIPF,527,30–60,8+,2,45.0,8,2,2
997,Agra,230085,90–120,14+,4,105.0,14,4,4
998,Mission: Red Planet,18258,60,10+,4–5,60.0,10,4,5


In [13]:
games_df['min_num_player'] = games_df['min_num_player'].astype(int)

In [14]:
games_df['max_num_player'] = games_df['max_num_player'].astype(int)

In [15]:
games = pd.read_csv('categories_transformed.csv', sep = '\t', encoding='utf-8')
games.drop(['Unnamed: 0',],axis=1,inplace=True)

In [16]:
games.drop(['full_url','url','id','categories'],axis=1,inplace=True)

In [17]:
games.head(3)

,name,Adventure,Exploration,Fantasy,Fighting,Miniatures,Environmental,Medical,Economic,Industry / Manufacturing,...,American Revolutionary War,Napoleonic,Childrens Game,Memory,American Civil War,Vietnam War,Math,Number,Trivia,nan
0,Gloomhaven,Yes,Yes,Yes,Yes,Yes,No,No,No,No,...,No,No,No,No,No,No,No,No,No,No
1,Pandemic Legacy: Season 1,No,No,No,No,No,Yes,Yes,No,No,...,No,No,No,No,No,No,No,No,No,No
2,Brass: Birmingham,No,No,No,No,No,No,No,Yes,Yes,...,No,No,No,No,No,No,No,No,No,No


In [18]:
games_df.drop(['id','play_time','player_age','user_players'],axis=1,inplace=True)

In [19]:
games_df

,name,avg_play_time,min_player_age,min_num_player,max_num_player
0,Gloomhaven,90.0,14,3,3
1,Pandemic Legacy: Season 1,60.0,12,4,4
2,Brass: Birmingham,90.0,14,3,4
3,Terraforming Mars,120.0,12,3,3
4,Twilight Imperium: Fourth Edition,360.0,14,6,6
...,...,...,...,...,...
995,Reavers of Midgard,90.0,12,4,4
996,GIPF,45.0,8,2,2
997,Agra,105.0,14,4,4
998,Mission: Red Planet,60.0,10,4,5


In [21]:
# top 20 categories for wich we are going to prepare data and train models
max_catg = ['Card Game', 'Fantasy', 'Economic', 'Fighting', 'Science Fiction',
       'Exploration', 'Adventure', 'Wargame', 'Medieval', 'City Building',
       'Miniatures', 'Dice', 'Deduction', 'Territory Building', 'Ancient',
       'Bluffing', 'Animals', 'Civilization', 'Nautical',
       'Industry / Manufacturing']

In [22]:
# collect in a dictionary games related to a particular categoty 
'''
key = Category
values = games related to the category 
'''
categorised_games = {}
for category in max_catg:

    categorised_games['{}'.format(category)] = games[games[category]=='Yes']['name']

In [23]:
categorised_games.keys()

dict_keys(['Card Game', 'Fantasy', 'Economic', 'Fighting', 'Science Fiction', 'Exploration', 'Adventure', 'Wargame', 'Medieval', 'City Building', 'Miniatures', 'Dice', 'Deduction', 'Territory Building', 'Ancient', 'Bluffing', 'Animals', 'Civilization', 'Nautical', 'Industry / Manufacturing'])

In [24]:
df = pd.DataFrame.from_dict(categorised_games)

In [25]:
final = pd.DataFrame()
df_lst = []
for column in df.columns:
    print(column)
    cat_series = df[column]
    cat_series = cat_series.dropna()
    cat_series = cat_series.to_frame()
    result = pd.DataFrame()
    category = cat_series.columns[0]
    cat_series = cat_series.rename(columns={column: 'Game'})
    result = pd.merge(cat_series, games_df, left_on='Game', right_on='name')
    result['Category'] = category
    df_lst += [result.copy()] 
final = pd.concat(df_lst)

Card Game
Fantasy
Economic
Fighting
Science Fiction
Exploration
Adventure
Wargame
Medieval
City Building
Miniatures
Dice
Deduction
Territory Building
Ancient
Bluffing
Animals
Civilization
Nautical
Industry / Manufacturing


In [26]:
final

,Game,name,avg_play_time,min_player_age,min_num_player,max_num_player,Category
0,Through the Ages: A New Story of Civilization,Through the Ages: A New Story of Civilization,120.0,14,3,3,Card Game
1,7 Wonders Duel,7 Wonders Duel,30.0,10,2,2,Card Game
2,Wingspan,Wingspan,55.0,10,3,3,Card Game
3,Arkham Horror: The Card Game,Arkham Horror: The Card Game,90.0,14,2,2,Card Game
4,Everdell,Everdell,60.0,10,3,3,Card Game
...,...,...,...,...,...,...,...
44,Container: 10th Anniversary Jumbo Edition!,Container: 10th Anniversary Jumbo Edition!,82.5,10,5,5,Industry / Manufacturing
45,Automania,Automania,75.0,12,4,4,Industry / Manufacturing
46,Coffee Roaster,Coffee Roaster,20.0,12,1,1,Industry / Manufacturing
47,Power Grid: Factory Manager,Power Grid: Factory Manager,60.0,12,4,4,Industry / Manufacturing


In [28]:
categorised_games['Adventure']

0                                   Gloomhaven
5                 Gloomhaven: Jaws of the Lion
12             War of the Ring: Second Edition
21                Arkham Horror: The Card Game
23                                     Nemesis
                        ...                   
940                                      Fangs
944    Fortune and Glory: The Cliffhanger Game
952            Ghost Fightin' Treasure Hunters
978                       Legacy of Dragonholt
981                        Claustrophobia 1643
Name: name, Length: 106, dtype: object

In [29]:
final[final['Category'] == 'Adventure']

,Game,name,avg_play_time,min_player_age,min_num_player,max_num_player,Category
0,Gloomhaven,Gloomhaven,90.0,14,3,3,Adventure
1,Gloomhaven: Jaws of the Lion,Gloomhaven: Jaws of the Lion,75.0,12,2,2,Adventure
2,War of the Ring: Second Edition,War of the Ring: Second Edition,165.0,14,2,2,Adventure
3,Arkham Horror: The Card Game,Arkham Horror: The Card Game,90.0,14,2,2,Adventure
4,Nemesis,Nemesis,135.0,14,4,4,Adventure
...,...,...,...,...,...,...,...
103,Fangs,Fangs,60.0,12,7,8,Adventure
104,Fortune and Glory: The Cliffhanger Game,Fortune and Glory: The Cliffhanger Game,165.0,10,3,3,Adventure
105,Ghost Fightin' Treasure Hunters,Ghost Fightin' Treasure Hunters,30.0,5,4,4,Adventure
106,Legacy of Dragonholt,Legacy of Dragonholt,529.5,10,1,1,Adventure


In [30]:
final[final['Game'] == 'Gloomhaven']

,Game,name,avg_play_time,min_player_age,min_num_player,max_num_player,Category
0,Gloomhaven,Gloomhaven,90.0,14,3,3,Fantasy
0,Gloomhaven,Gloomhaven,90.0,14,3,3,Fighting
0,Gloomhaven,Gloomhaven,90.0,14,3,3,Exploration
0,Gloomhaven,Gloomhaven,90.0,14,3,3,Adventure
0,Gloomhaven,Gloomhaven,90.0,14,3,3,Miniatures


In [31]:
# dictionary of European contries was adjusted, use file all_rating_processed_adj_dict.csv insread 
data = pd.read_csv(r'C:/Users/Armit/uni_pass/Scripts/Project/all_rating_processed_adj_dict.csv', sep = '\t', encoding='utf-8')

In [32]:
data.drop(['Unnamed: 0','Location','file','Country'],axis = 1, inplace=True)

In [33]:
data.head(3)

,Game_name,Gamer,Date,Rating,Continent
0,Gloomhaven,guile678,Mar2020,8.70000,Undefined
1,Gloomhaven,purplephoenixgames,Mar2020,-0.00001,NorthAmerica
2,Gloomhaven,oneover,Mar2020,8.00000,Undefined


In [34]:
# exclude not provided rating , whichj we earlier marked as -0.00001
data = data[data['Rating'] >0]

In [35]:
grouped_rating = data.groupby(['Game_name','Continent'])['Rating'].mean()

In [36]:
grouped_rating = grouped_rating.reset_index()

In [37]:
grouped_rating[grouped_rating['Game_name'] == 'Gloomhaven']

,Game_name,Continent,Rating
1122,Gloomhaven,Europe,8.767816
1123,Gloomhaven,NorthAmerica,8.501645
1124,Gloomhaven,Undefined,8.839919


In [38]:
final_2 = final.copy(deep=True)

In [39]:
final_2 = final_2.merge(grouped_rating,left_on='Game', right_on='Game_name')

In [40]:
final_2.drop(['name','Game_name'],axis = 1, inplace=True)

In [41]:
# check dataset. 
# data is almost ready, need to transform categorical features, data types, and excude Undefined locations 
final_2[final_2['Game'] == 'Gloomhaven']

,Game,avg_play_time,min_player_age,min_num_player,max_num_player,Category,Continent,Rating
1923,Gloomhaven,90.0,14,3,3,Fantasy,Europe,8.767816
1924,Gloomhaven,90.0,14,3,3,Fantasy,NorthAmerica,8.501645
1925,Gloomhaven,90.0,14,3,3,Fantasy,Undefined,8.839919
1926,Gloomhaven,90.0,14,3,3,Fighting,Europe,8.767816
1927,Gloomhaven,90.0,14,3,3,Fighting,NorthAmerica,8.501645
1928,Gloomhaven,90.0,14,3,3,Fighting,Undefined,8.839919
1929,Gloomhaven,90.0,14,3,3,Exploration,Europe,8.767816
1930,Gloomhaven,90.0,14,3,3,Exploration,NorthAmerica,8.501645
1931,Gloomhaven,90.0,14,3,3,Exploration,Undefined,8.839919
1932,Gloomhaven,90.0,14,3,3,Adventure,Europe,8.767816


In [42]:
data_lr = final_2.copy(deep=True)

In [43]:
data_lr.reset_index(inplace = True)

In [44]:
data_lr.drop(['index','Game'],axis = 1, inplace=True)

In [45]:
data_lr

,avg_play_time,min_player_age,min_num_player,max_num_player,Category,Continent,Rating
0,120.0,14,3,3,Card Game,Europe,8.568642
1,120.0,14,3,3,Card Game,NorthAmerica,8.378970
2,120.0,14,3,3,Card Game,Undefined,8.570000
3,120.0,14,3,3,Economic,Europe,8.568642
4,120.0,14,3,3,Economic,NorthAmerica,8.378970
...,...,...,...,...,...,...,...
6289,67.5,10,3,4,Industry / Manufacturing,NorthAmerica,7.135505
6290,67.5,10,3,4,Industry / Manufacturing,Undefined,7.332593
6291,105.0,14,4,4,Industry / Manufacturing,Europe,7.348533
6292,105.0,14,4,4,Industry / Manufacturing,NorthAmerica,7.357455


In [46]:
# excluding Undefined locations from dataset
data_lr = data_lr[data_lr['Continent'] != 'Undefined']

In [47]:
X = data_lr[['avg_play_time','min_player_age','min_num_player','max_num_player','Category','Continent']]
y = data_lr['Rating']

In [48]:
X['min_num_player'] = X['min_num_player'].astype(str)

<ipython-input-48-9a1fdfa0e7c5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['min_num_player'] = X['min_num_player'].astype(str)


In [49]:
X['max_num_player'] = X['max_num_player'].astype(str)

<ipython-input-49-07ed1cd60742>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['max_num_player'] = X['max_num_player'].astype(str)


In [50]:
X['min_player_age'] = X['min_player_age'].astype(str)

<ipython-input-50-3f45772977f1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['min_player_age'] = X['min_player_age'].astype(str)


In [51]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4196 entries, 0 to 6292
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   avg_play_time   4196 non-null   float64
 1   min_player_age  4196 non-null   object 
 2   min_num_player  4196 non-null   object 
 3   max_num_player  4196 non-null   object 
 4   Category        4196 non-null   object 
 5   Continent       4196 non-null   object 
dtypes: float64(1), object(5)
memory usage: 229.5+ KB


In [52]:
# perform one hot encode of categorical features 
X = pd.get_dummies(X, drop_first=True)
X

,avg_play_time,min_player_age_12,min_player_age_14,min_player_age_16,min_player_age_18,min_player_age_3,min_player_age_4,min_player_age_5,min_player_age_6,min_player_age_8,...,Category_Fantasy,Category_Fighting,Category_Industry / Manufacturing,Category_Medieval,Category_Miniatures,Category_Nautical,Category_Science Fiction,Category_Territory Building,Category_Wargame,Continent_NorthAmerica
0,120.0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,120.0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,120.0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,120.0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
6,120.0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6286,105.0,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
6288,67.5,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
6289,67.5,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
6291,105.0,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [53]:
x_train, x_test, y_train, y_test = train_test_split(X, y, train_size=0.75, test_size=0.25, random_state=12345)

In [54]:
x_train = x_train.dropna()

#### Linear regression model

In [68]:
model = LinearRegression()
model.fit(x_train,y_train)
predicted_lr = model.predict(x_test) 
r2 = r2_score(y_test, predicted)
r2

0.38066984407721927

In [69]:
mean_absolute_error(y_test, predicted_lr)

0.284518172225706

In [70]:
mean_squared_error(y_test, predicted_lr, squared = True)

0.1297102689261284

In [57]:
feature_names = list(x_train.columns)

In [58]:
eli5.show_weights(model, top=40, feature_names = feature_names)

Weight?,Feature
+7.375,<BIAS>
+1.167,min_player_age_18
+0.659,max_num_player_6
+0.531,max_num_player_10
+0.522,max_num_player_3
+0.484,max_num_player_8
+0.475,max_num_player_5
+0.468,max_num_player_2
+0.459,max_num_player_4
+0.436,max_num_player_7


#### RandomForest model

In [71]:
regr = RandomForestRegressor(random_state=0,oob_score=True)

In [72]:
regr.fit(x_train, y_train)

RandomForestRegressor(oob_score=True, random_state=0)

In [73]:
regr.oob_score_

0.3710875906446439

In [74]:
predicted_forest = regr.predict(x_test)

In [63]:
eli5.explain_weights(regr, top=40, feature_names = feature_names)

Weight,Feature
0.3897 ± 0.0329,avg_play_time
0.0480 ± 0.0094,Continent_NorthAmerica
0.0428 ± 0.0161,min_num_player_2
0.0387 ± 0.0139,min_player_age_12
0.0288 ± 0.0096,min_player_age_14
0.0240 ± 0.0217,min_num_player_4
0.0214 ± 0.0080,Category_Fantasy
0.0203 ± 0.0128,max_num_player_4
0.0201 ± 0.0083,min_player_age_8
0.0200 ± 0.0063,Category_Economic


In [75]:
mean_absolute_error(y_test, predicted)

0.24575908555043838

In [76]:
mean_squared_error(y_test, predicted, squared = True)

0.10668602164308388

In [65]:
import eli5
from eli5.sklearn import PermutationImportance

In [66]:
perm = PermutationImportance(regr).fit(x_test, y_test)
eli5.show_weights(perm,top=50,feature_names = feature_names)

Weight,Feature
0.5486 ± 0.1020,avg_play_time
0.1396 ± 0.0257,min_player_age_14
0.1318 ± 0.0214,min_num_player_2
0.0770 ± 0.0259,min_player_age_12
0.0604 ± 0.0071,max_num_player_2
0.0430 ± 0.0092,min_num_player_5
0.0209 ± 0.0123,min_num_player_4
0.0206 ± 0.0057,min_player_age_8
0.0181 ± 0.0017,min_player_age_18
0.0139 ± 0.0125,max_num_player_4
